<a href="https://colab.research.google.com/github/yorkjong/rs_rating/blob/main/notebooks/rsm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with RSM (Mansfield Relative Strength) Rating

### Install and Config

#### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/rs_rating.git"
%pip install requests-cache

  Cloning https://github.com/yorkjong/rs_rating.git to /tmp/pip-req-build-bta6drxw
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/rs_rating.git /tmp/pip-req-build-bta6drxw
  Resolved https://github.com/yorkjong/rs_rating.git to commit 07c6fe7f8e18d8fca90c33a3c382832478be065d
  Preparing metadata (setup.py) ... done
  Created wheel for rs_rating: filename=rs_rating-1.0-py3-none-any.whl size=26014 sha256=c24cd9d19622ab59944c20fb928b5c47aafe998670b15374845bbdef4f7d3285
  Stored in directory: /tmp/pip-ephem-wheel-cache-ivz5tyd2/wheels/6c/e3/e2/d823f25048beb6ccc0ebc402d5210ce344ec7a829a17558710
Successfully built rs_rating
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.9 MB/s eta 0:00:00


#### Setup and Configuration

In [2]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [6]:
# @title Rank Function

import os
from datetime import datetime

from rs_rating import rsm
from rs_rating.stock_indices import get_tickers


def remove_failed_tickers(tickers):
    delisted = ['BRK.B', 'LEN.B', 'BF.B', 'UHAL.B', 'BF.A', 'CWEN.A', 'HEI.A']
    invalid = ['GEV', 'SOLV', 'VLTO', 'SW', 'ARM', 'CART', 'AS', 'BIRK', 'VSTS','LOAR', 'ALAB','GRAL', 'SEG']
    invalid += ['00945B.TW', '6928.TW', '6914.TW', '6771.TW', '00944.TW', '8162.TW', '1563.TW', '00946.TW', '00941.TW', '6423.TW', '00940.TW', '00939.TW', '4949.TW', '00943.TW', '8487.TW', '6794.TW', '6949.TW', '4771.TW']
    invalid += ['00936.TW', '6805.TW', '2254.TW', '6658.TW', '00935.TW', '6592B.TW', '6526.TW', '6906.TW', '4736.TW', '00636K.TW', '6968.TWO', '4442.TWO', '6534.TW', '6901.TW', '00934.TW', '00657K.TW', '6472.TW', '2258.TW', '6916.TW', '2762.TW', '6933.TW']
    invalid += ['02001R.TW', '020031.TW', '020039.TW', '020016.TW', '02001L.TW', '020019.TW', '020028.TW', '020020.TW', '02001S.TW', '020018.TW', '020038.TW', '020034.TW', '020011.TW', '020030.TW', '020012.TW', '020036.TW', '020029.TW', '020000.TW', '020015.TW', '020037.TW']
    invalid += ['6890.TW', '00951.TW', '3150.TW', '6957.TW', '00947.TW', '00949.TW']
    invalid += ['6838.TW', '00953B.TW', '00956.TW', '00954.TW']
    return list(set(tickers) - set(delisted) - set(invalid))

def rank(code, period='2y', ma="EMA", tickers_getter=get_tickers,
         ticker_ref='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    #tickers = remove_failed_tickers(tickers)

    rank_stock = rsm.ranking(tickers, period=period,
                             ticker_ref=ticker_ref, ma=ma)
    if rank_stock.empty:
        print("Not enough data to generate rankings.")
        return

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    filename = f'{code}_stocks_{period}_{ma}_{today}.csv'
    #github.upload_df_as_csv(filename, rank_stock)
    rank_stock.to_csv(os.path.join(out_dir, filename), index=False)
    print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock

### RS Rating and Ranking

In [7]:
source = "S&P 500" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "Russell 1000", "Russell 2000", "PHLX Semiconductor", "U.S. Listed Stocks"]
ma = "SMA" # @param ["EMA","SMA"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'Russell 2000': 'RUT',
    'PHLX Semiconductor': 'SOX',
    'U.S. Listed Stocks': 'U.S.Listed',
}

stock_df = rank(code_from_name[source], ma=ma)
data_table.DataTable(stock_df, include_index=False, num_rows_per_page=20)



[**********************100%**********************]  501 of 501 info downloaded


[*********************100%***********************]  502 of 502 completed
ERROR:yfinance:
4 Failed downloads:
ERROR:yfinance:['AMTM']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']")
ERROR:yfinance:['SW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")
ERROR:yfinance:['GEV', 'SOLV']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']")


Num of downloaded stocks: 502
[**********************100%**********************]  501 of 501 financials downloaded




***
Your "SPX_stocks_2y_SMA_20241006.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (P),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
474,VST,Utilities,Utilities - Independent Power Producers,90.41,66.55,30.49,54.88,74.38,23.63,100.0,138.41,92.49,85.09,1.68,36.36,1.36,-0.97,39.657,101.77
374,PLTR,Technology,Software - Infrastructure,51.87,42.00,29.34,15.48,20.83,17.87,99.8,40.01,33.01,26.83,0.99,30.20,0.17,8.67,1.127,235.35
84,CEG,Utilities,Utilities - Renewable,51.26,38.41,11.35,25.82,44.31,9.69,99.6,285.52,210.52,202.76,1.16,33.92,7.51,-8.26,74.038,38.02
344,NVDA,Technology,Semiconductors,30.79,29.04,35.55,66.47,60.12,22.84,99.4,124.92,116.96,108.61,0.90,23.47,2.14,24.01,3.910,58.37
234,HWM,Industrials,Aerospace & Defense,29.73,27.70,31.17,16.55,16.22,6.21,99.2,102.03,95.82,82.44,0.96,11.91,2.25,5.26,17.300,45.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,WBA,Healthcare,Pharmaceutical Retailers,-57.43,-56.51,-57.94,-52.22,-28.31,-20.31,0.2,8.72,9.71,14.12,1.11,503.12,-6.74,-0.84,168.626,NaN
31,AMTM,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.75,NaN,-4.14,NaN,15.76
201,GEV,Utilities,Utilities - Renewable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,641.16,3.96,7.17,122.901,67.07
415,SOLV,Healthcare,Health Information Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-153.68,2.34,0.19,47.519,28.95


In [10]:
# @title Display industry_df

from rs_rating.ranking_utils import groupby_industry, append_percentile

rs_columns = ['RS (%)', '1 Week Ago',
              '1 Month Ago', '3 Months Ago', '6 Months Ago', '9 Months Ago']
columns =  ['Sector', 'Ticker'] + rs_columns
industry_df = groupby_industry(stock_df, columns, key='RS (%)')

industry_df = industry_df.sort_values(by='RS (%)', ascending=False)
industry_df = append_percentile(industry_df, rs_columns)
data_table.DataTable(industry_df, include_index=False, num_rows_per_page=20)

,Industry,Sector,Ticker,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Pctl (RS (%)),Pctl (1 Week Ago),Pctl (1 Month Ago),Pctl (3 Months Ago),Pctl (6 Months Ago),Pctl (9 Months Ago)
108,Utilities - Independent Power Producers,Utilities,"VST,NRG",59.84,45.96,26.84,41.09,54.92,23.74,99,99,99,99,99,99
112,Utilities - Renewable,Utilities,"CEG,GEV",51.26,38.41,11.35,25.82,44.31,9.69,98,98,94,98,98,91
42,"Furnishings, Fixtures & Appliances",Consumer Cyclical,MHK,17.65,22.07,24.12,-4.06,13.44,-6.14,97,97,98,69,91,35
69,Oil & Gas Midstream,Energy,"TRGP,WMB,KMI,OKE",14.90,7.16,10.63,4.98,2.38,-4.23,96,86,93,92,66,44
43,Gold,Basic Materials,NEM,14.18,15.77,17.06,-5.80,-22.95,-14.65,96,96,96,63,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,Apparel Retail,Consumer Cyclical,"TJX,ROST,LULU",-14.59,-11.81,-13.02,-11.60,-5.04,10.55,4,13,10,27,41,95
29,Discount Stores,Consumer Defensive,"WMT,COST,TGT,DG,DLTR",-14.94,-14.03,-12.54,-6.07,-2.13,-8.45,3,6,11,61,49,28
46,Healthcare Plans,Healthcare,"UNH,CI,CNC,ELV,CVS,MOH,HUM",-16.60,-13.34,-10.40,-16.51,-7.32,-5.42,3,7,15,10,32,37
10,Auto Parts,Consumer Cyclical,"BWA,GPC,APTV,LKQ",-17.23,-14.44,-17.71,-23.74,-17.40,-19.79,2,5,4,2,6,4
